Handling different columns requiring different types of encoding is quite difficult. This is because when we do encoding it creates a numpy array, then we have merge them all which is quite cumbersome and inefficient